# Programmatic Indicators Analysis

This notebook analyzes programmatic indicators like families assisted and security cameras.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Create charts directory
os.makedirs("../public/charts", exist_ok=True)

print("✅ Environment ready")

In [ ]:
# Load indicators data
try:
    df_ind = pd.read_csv("../data/processed/indicators_all_years.csv")
    print(f"✅ Loaded indicators data with {len(df_ind)} rows")
    
    # Show data structure
    print(f"\n📋 Columns: {list(df_ind.columns)}")
    print(f"\n📊 Indicator types: {df_ind['indicator'].unique()}")
    
except Exception as e:
    print(f"❌ Error loading indicators data: {e}")
    df_ind = None

In [ ]:
# Analyze security cameras data
if df_ind is not None:
    df_cam = df_ind[df_ind['indicator'] == 'security_cameras'].copy()
    
    if len(df_cam) > 0:
        # Create period column
        df_cam['period'] = df_cam['year'].astype(str) + '-' + df_cam['quarter']
        df_cam = df_cam.sort_values(['year', 'quarter'])
        
        # Create line chart
        fig = go.Figure()
        
        # Add planned line
        if 'planned' in df_cam.columns:
            fig.add_trace(go.Scatter(
                x=df_cam['period'],
                y=df_cam['planned'],
                mode='lines+markers',
                name='Planned',
                line=dict(color='blue', width=3),
                marker=dict(size=8)
            ))
        
        # Add executed line
        if 'executed' in df_cam.columns:
            fig.add_trace(go.Scatter(
                x=df_cam['period'],
                y=df_cam['executed'],
                mode='lines+markers',
                name='Executed',
                line=dict(color='green', width=3),
                marker=dict(size=8)
            ))
        
        fig.update_layout(
            title='Security Cameras: Planned vs Executed (2022–2025)',
            xaxis_title='Period',
            yaxis_title='Number of Cameras',
            hovermode='x unified',
            height=600,
            showlegend=True
        )
        
        fig.show()
        fig.write_html("../public/charts/security_cameras_timeline.html")
        print("✅ Generated security cameras timeline chart")
        
        # Key insight
        latest_executed = df_cam['executed'].iloc[-1] if len(df_cam) > 0 and 'executed' in df_cam.columns else 0
        latest_planned = df_cam['planned'].iloc[-1] if len(df_cam) > 0 and 'planned' in df_cam.columns else 1
        if latest_planned > 0:
            execution_rate = (latest_executed / latest_planned) * 100
            print(f"💡 Key insight: Latest period execution rate: {execution_rate:.1f}%")
    else:
        print("❌ No security cameras data found")
else:
    print("❌ Cannot analyze security cameras data")

In [ ]:
# Analyze families assisted data
if df_ind is not None:
    df_fam = df_ind[df_ind['indicator'] == 'families_assisted'].copy()
    
    if len(df_fam) > 0:
        # Create period column
        df_fam['period'] = df_fam['year'].astype(str) + '-' + df_fam['quarter']
        df_fam = df_fam.sort_values(['year', 'quarter'])
        
        # Create bar chart
        fig2 = px.bar(
            df_fam,
            x='period',
            y='executed',
            title='Families Assisted by Period',
            labels={'executed': 'Number of Families', 'period': 'Period'},
            color='executed',
            color_continuous_scale='blues'
        )
        
        fig2.update_layout(
            xaxis_title='Period',
            yaxis_title='Number of Families',
            height=500
        )
        
        fig2.show()
        fig2.write_html("../public/charts/families_assisted_timeline.html")
        print("✅ Generated families assisted timeline chart")
        
        # Total families assisted
        total_families = df_fam['executed'].sum() if 'executed' in df_fam.columns else 0
        print(f"📊 Total families assisted: {total_families:,.0f}")
    else:
        print("❌ No families assisted data found")
else:
    print("❌ Cannot analyze families assisted data")